In [1]:
import os
import math
import torch
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import models
from analyze import XRayDataset, load_model
from collections import defaultdict

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataset, loader, model = load_model(device)
num_classes = len(dataset.classes)

criterion = nn.CrossEntropyLoss(weight=dataset.class_weights.to(device))

images_per_class = defaultdict(lambda: [])

j = 0
count = len(dataset)
for inputs, labels, filenames in loader:
    with torch.set_grad_enabled(False):
        j += 1
        if j % 100 == 0:
            print(f'{j}/{count}')
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)

        _, preds = torch.max(outputs, 1)

        for i in range(len(labels)):
            label = labels[i:i+1]
            pred = preds[i]
            output = outputs[i:i+1]
            loss = criterion(output, label)
            
            images_per_class[labels[i].item()].append({
                'image': filenames[i],
                'correct': bool(label == pred),
                'loss': loss.item()
            })

100/7992
200/7992
300/7992
400/7992
500/7992
600/7992
700/7992
800/7992
900/7992
1000/7992
1100/7992
1200/7992
1300/7992
1400/7992
1500/7992
1600/7992
1700/7992
1800/7992
1900/7992
2000/7992
2100/7992
2200/7992
2300/7992
2400/7992
2500/7992
2600/7992
2700/7992
2800/7992
2900/7992
3000/7992
3100/7992
3200/7992
3300/7992
3400/7992
3500/7992
3600/7992
3700/7992
3800/7992
3900/7992
4000/7992
4100/7992
4200/7992
4300/7992
4400/7992
4500/7992
4600/7992
4700/7992
4800/7992
4900/7992
5000/7992
5100/7992
5200/7992
5300/7992
5400/7992
5500/7992
5600/7992
5700/7992
5800/7992
5900/7992
6000/7992
6100/7992
6200/7992
6300/7992
6400/7992
6500/7992
6600/7992
6700/7992
6800/7992
6900/7992
7000/7992
7100/7992
7200/7992
7300/7992
7400/7992
7500/7992
7600/7992
7700/7992
7800/7992
7900/7992


In [3]:
import json
best_ten = {}

for label, images in images_per_class.items():
    images = filter(lambda i: i['correct'], images)
    images = sorted(images, key=lambda i: i['loss'])
    best_ten[dataset.classes[label]] = images[:10]

with open('best10.json', 'w') as f:
    json.dump(best_ten, f, indent=2)